# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846839659784                   -0.70    4.8
  2   -7.852314745433       -2.26       -1.53    1.0
  3   -7.852646237141       -3.48       -2.52    3.2
  4   -7.852646677844       -6.36       -3.36    2.2
  5   -7.852646685958       -8.09       -4.73    1.2
  6   -7.852646686724       -9.12       -5.25    3.2
  7   -7.852646686730      -11.25       -6.17    1.5
  8   -7.852646686730      -12.68       -7.69    2.2
  9   -7.852646686730   +  -14.75       -7.66    2.2
 10   -7.852646686730      -14.57       -8.59    2.0
 11   -7.852646686730      -15.05       -9.58    2.0
 12   -7.852646686730   +  -14.27      -10.08    3.2
 13   -7.852646686730      -14.75      -11.00    1.2
 14   -7.852646686730      -14.75      -11.74    1.8
 15   -7.852646686730      -14.75      -12.51    2.5


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846777838224                   -0.70           4.5
  2   -7.852552923690       -2.24       -1.63   0.80    2.2
  3   -7.852639943077       -4.06       -2.72   0.80    1.0
  4   -7.852646499093       -5.18       -3.39   0.80    2.0
  5   -7.852646682783       -6.74       -4.34   0.80    1.8
  6   -7.852646686622       -8.42       -4.94   0.80    2.0
  7   -7.852646686722      -10.00       -5.78   0.80    1.5
  8   -7.852646686730      -11.14       -6.40   0.80    1.8
  9   -7.852646686730      -12.87       -7.50   0.80    1.5
 10   -7.852646686730      -13.88       -7.73   0.80    2.5
 11   -7.852646686730   +    -Inf       -8.86   0.80    1.0
 12   -7.852646686730   +  -15.05       -9.87   0.80    2.5
 13   -7.852646686730   +  -15.05      -10.34   0.80    2.0
 14   -7.852646686730   +    -Inf      -11.42   0.80    1.8
 15   -7.852646686730   +    -Inf      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.402602e+01     3.402783e+00
 * time: 0.40276217460632324
     1     1.437095e+00     1.815241e+00
 * time: 0.617279052734375
     2    -1.625675e+00     1.806442e+00
 * time: 0.640296220779419
     3    -3.787937e+00     1.584054e+00
 * time: 0.6730551719665527
     4    -5.327125e+00     1.421487e+00
 * time: 0.7060911655426025
     5    -6.760763e+00     9.978959e-01
 * time: 0.7387070655822754
     6    -6.890517e+00     1.290125e+00
 * time: 0.7613282203674316
     7    -7.518939e+00     7.262915e-01
 * time: 0.7842061519622803
     8    -7.612943e+00     8.648780e-01
 * time: 0.8069131374359131
     9    -7.755586e+00     5.540441e-01
 * time: 0.8295080661773682
    10    -7.796804e+00     4.940534e-01
 * time: 0.8522422313690186
    11    -7.819159e+00     1.412873e-01
 * time: 0.8748900890350342
    12    -7.833495e+00     4.791729e-02
 * time: 0.9740312099456787
    13    -7.843584e+00     5.498711e-02
 * time: 0.99707007408

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846868660166                   -0.70    4.8
  2   -7.852320385317       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686708                   -2.54
  2   -7.852646686730      -10.67       -5.98
  3   -7.852646686730   +  -14.75      -12.59


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.1599091147754445e-13
|ρ_newton - ρ_scfv| = 7.859234430161312e-13
|ρ_newton - ρ_dm|   = 1.4023747666911717e-10
